# Visualization of the COPC using Lonboard

In [1]:
import geopandas as gpd
from lonboard import viz
import pdal
import pandas as pd

from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
import httpx
import json
from palettable.colorbrewer.sequential import YlGnBu_7

(PDAL Error) Can't load library /opt/conda/envs/pangeo/lib/libpdal_plugin_reader_draco.so: Failed to load "/opt/conda/envs/pangeo/lib/libpdal_plugin_reader_draco.so": libdraco.so.8: cannot open shared object file: No such file or directory(PDAL Error) Can't load library /opt/conda/envs/pangeo/lib/libpdal_plugin_writer_draco.so: Failed to load "/opt/conda/envs/pangeo/lib/libpdal_plugin_writer_draco.so": libdraco.so.8: cannot open shared object file: No such file or directory(PDAL Error) Can't load library /opt/conda/envs/pangeo/lib/libpdal_plugin_reader_draco.so: Failed to load "/opt/conda/envs/pangeo/lib/libpdal_plugin_reader_draco.so": libdraco.so.8: cannot open shared object file: No such file or directory(PDAL Error) Can't load library /opt/conda/envs/pangeo/lib/libpdal_plugin_writer_draco.so: Failed to load "/opt/conda/envs/pangeo/lib/libpdal_plugin_writer_draco.so": libdraco.so.8: cannot open shared object file: No such file or directory

In [2]:
path = "my_scripts/minord/ALS_ground_copc/outputs/norm_copc/norm_copc_chunk_623000_1012200.copc.laz"

In [3]:
dir_path = "../my_scripts/minord/ALS_ground_copc/outputs/norm_copc/norm_copc_chunk_623400_*.copc.laz"
out_file = "../my_scripts/minord/ALS_ground_copc/outputs/norm_copc/norm_copc_merged_623400.copc.laz"

In [4]:
pipeline = (pdal.Reader.copc(filename=dir_path) | pdal.Filter.merge() | pdal.Writer.copc(out_file))

pipeline.execute()

2255837

### Read the merged COPC file

In [7]:
pipeline2 = (pdal.Reader.copc(filename=out_file) | pdal.Filter.reprojection(out_srs="epsg:4326"))
pipeline2.execute()

2255837

In [8]:
# Getting array values
arr_values = pipeline2.arrays

In [9]:
arr_values[0].shape

(2255837,)

In [10]:
df = pd.DataFrame(arr_values[0])

In [11]:
df.head()

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,ScanChannel,ClassFlags,TargetThick,DevRatio,Zref
0,-79.87,9.14,12.16,58,1,3,0,0,1,34.127998,1,22,3.691544e+08,0,0,0.360,1.1,39.56
1,-79.87,9.14,12.47,46,1,3,0,0,1,34.133999,1,22,3.691544e+08,0,0,0.112,1.1,39.87
2,-79.87,9.14,13.04,31,1,2,0,0,1,34.374001,1,22,3.691544e+08,0,0,0.144,0.7,40.91
3,-79.87,9.14,12.55,44,1,1,0,0,1,34.344002,1,22,3.691544e+08,0,0,0.296,1.5,40.41
4,-79.87,9.14,13.22,48,1,1,0,0,1,34.313999,1,22,3.691544e+08,0,0,0.304,1.6,41.04


In [12]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X, df.Y), crs="EPSG:4326")

In [13]:
gdf.head()

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,ScanChannel,ClassFlags,TargetThick,DevRatio,Zref,geometry
0,-79.87,9.14,12.16,58,1,3,0,0,1,34.127998,1,22,3.691544e+08,0,0,0.360,1.1,39.56,POINT (-79.87000 9.14000)
1,-79.87,9.14,12.47,46,1,3,0,0,1,34.133999,1,22,3.691544e+08,0,0,0.112,1.1,39.87,POINT (-79.87000 9.14000)
2,-79.87,9.14,13.04,31,1,2,0,0,1,34.374001,1,22,3.691544e+08,0,0,0.144,0.7,40.91,POINT (-79.87000 9.14000)
3,-79.87,9.14,12.55,44,1,1,0,0,1,34.344002,1,22,3.691544e+08,0,0,0.296,1.5,40.41,POINT (-79.87000 9.14000)
4,-79.87,9.14,13.22,48,1,1,0,0,1,34.313999,1,22,3.691544e+08,0,0,0.304,1.6,41.04,POINT (-79.87000 9.14000)


### Define a Point Cloud Layer

In [ ]:
titiler_endpoint = "https://titiler.maap-project.org"

In [129]:
point_layer = ScatterplotLayer.from_geopandas(gdf)

In [ ]:
point_layer.get_fill_color = apply_continuous_cmap(gdf["Intensity"], YlGnBu_7, alpha=0.7)

### Read DTM Layer

In [ ]:
dtm_layer_url = ""

In [ ]:
dtm_tile_url = "https://titiler.maap-project.org/cog/tiles/{z}/{x}/{y}?url=s3://maap-ops-workspace/shared/alexdevseed/geotrees/meta_chm_032221130_cog.tif&rescale=0,60&colormap_name=viridis"

In [ ]:
r_dtm = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": dtm_layer_url,
    }
).json()

bounds_dtm = r_dtm["bounds"]
print(bounds_dtm)

In [ ]:
dtm_layer = BitmapTileLayer(
    data=dtm_tile_url,
    tile_size=148,
    max_requests=-1,
    min_zoom=3,
    max_zoom=18,
    extent=bounds_dtm
)

### Read Final AGB Product Layer

In [ ]:
agb_product_url = ""

In [ ]:
agb_product_tile_url = "https://titiler.maap-project.org/cog/tiles/{z}/{x}/{y}?url=s3://maap-ops-workspace/shared/alexdevseed/geotrees/meta_chm_032221130_cog.tif&rescale=0,60&colormap_name=viridis"

In [ ]:
r_agb = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": agb_product_url,
    }
).json()

bounds_agb = r_agb["bounds"]
print(bounds_agb)

In [ ]:
agb_product_layer = BitmapTileLayer(
    data=agb_product_tile_url,
    tile_size=148,
    max_requests=-1,
    min_zoom=3,
    max_zoom=18,
    extent=bounds
)

### Create a Map Object for all the layers

In [130]:
m = Map([point_layer, dtm_layer, agb_product_layer])
m

Map(layers=[ScatterplotLayer(table=pyarrow.Table
X: float
Y: float
Z: float
Intensity: uint8
ReturnNumber: uin…